## Import hfo data from .mat files and process it

### Check WD (change if necessary) and file loading

In [ ]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/seeg_data/clinical" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/seeg_data/clinical"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

# Choose the patient
PATIENT_LABEL = 'csl' # 'csl'
PATH_TO_FILE = f'patients/{PATIENT_LABEL}/' # 'patients/csl/'   # This is needed if the WD is not the same as the file location

: 

The `.mat` files are not uploaded to GitHub. As such, they must be taken from GDrive and moved to the `clinical` folder

In [ ]:
import scipy.io as sio
import numpy as np

# Load the data
data = sio.loadmat(f'{PATH_TO_FILE}signal.mat')

# Print the data structure
print(data.keys())

: 

# Print the content of the .mat file

## General experiment information

In [ ]:
sampling_rate = data['sr'][0][0]
input_duration = data['duration'][0][0]
num_samples = data['samples'][0][0]

print(f"sr: {sampling_rate}")  # Sampling rate

print(f"duration: {input_duration}")

print(f"samples: {num_samples}")

: 

- The **sampling rate** is 2048 Hz, which means that 2048 samples are recorded per second (for each electrode).
- The **duration** of the experiment is ~63.1s.
- There is a **total of 129239 samples** (sampling_rate * duration) per electrode.

## Channel information

In [ ]:
num_channels = data['channels'].shape[1]

print(f"Shape of channels: {data['channels'].shape}")

print(f"channels: {data['channels']}")

print("=====================================\n\n\n")

print(f"channel_types: {data['channel_types']}")

: 

This recording has 86 channels, which correspond to 86 electrodes**. 

The channels are divided into **X groups of Y channels each**. Each **group** corresponds to a **different brain region**:
- **Group 1**: OT'8
- **Group 2**: B'1
- **Group 3**: GPH'2
- **Group 4**: A'1
- **Group 5**: I6
- **Group 6**: PM6
- **Group 7**: PM10
- **Group 8**: CR5

## Markers information
The `.mat` file does not contain relevant info about the markers. We will load it from a `.csv` file

In [ ]:
if 'markers' not in data.keys():
    print("No markers in the data")
else:
    print("Shape of markers: ", data['markers'].shape)

    print(f"markers: {data['markers']}")

: 

## Read Markers data from the csv file

In [ ]:
# Read csv file 
import pandas as pd

markers_csv = pd.read_csv(f'{PATH_TO_FILE}markers_for_test.csv')

# print(markers_csv)

# Remove marker column- `Value`: I'm not sure what it refers to
markers_csv = markers_csv.drop(columns=['Value'])

print(markers_csv)

: 

## Create a column for the channel indices

## Map channel names to indices

In [ ]:
channel_idx_map = {}

for idx in range(num_channels):
    # print(f"Channel {idx}: {data['channels'][0][idx][0]}")
    channel_idx_map[data['channels'][0][idx][0]] = idx
    

print("channel_idx_map: ", channel_idx_map)

: 

### Get the channel idx from the channel names

In [ ]:
# Map the channel names to the channel index. If mapping is not found, delete the row
markers_csv['channel_idx'] = markers_csv['Target'].map(channel_idx_map)

# Drop rows with NaN values in the channel_idx column (TODO: Check why some channel names are not valid)
markers_csv = markers_csv.dropna(subset=['channel_idx'])

# Convert the channel_idx column to int
markers_csv['channel_idx'] = markers_csv['channel_idx'].astype(int)

# Drop the column with the channel name now that we have the index
markers_csv = markers_csv.drop(columns=['Target'])

markers_csv

: 

###  Convert the Position and Duration column from seconds to milliseconds

In [ ]:
markers_csv['Pos'] = markers_csv['Pos'].map(lambda x: x*1000)
markers_csv['Duration'] = markers_csv['Duration'].map(lambda x: x*1000)   # Should be 0 already in the synthetic dataset

print(markers_csv)

: 

### Create a numpy array from the csv file

In [ ]:
markers_npy = markers_csv.to_numpy()

markers_npy

: 

## Join all the markers data into a single structured datatype
We want to group the markers by channel, so the shape of the structure datatype = (num_channels, [events])

The structured datatype will have the following fields:
- `label`: the type of event that occurred (can be a combination of 'Spike', 'Ripple' and 'Fast-Ripple')
- `position`: the position of the marker in milliseconds
- `duration`: the duration of the event in milliseconds

The index of each row corresponds to the channel the event is related to

In [ ]:
from utils.io import preview_np_array

# Create an empty np array to store the markers
# markers_arr = [ np.empty(shape=(1), dtype=[('label', 'U64'), ('position', np.float32), ('duration', np.float32)]) for _ in range(num_channels)]
markers_arr = np.empty(num_channels).tolist()
for idx in range(num_channels):
    markers_arr[idx] = [np.empty(shape=(0), dtype=[('label', 'U64'), ('position', np.float32), ('duration', np.float32)])]

print("markers_arr: ", markers_arr)

: 

In [ ]:
# Iterate all the marked events and add them to markers_arr
for rowIdx in range(len(markers_npy)):    
    currObj = markers_npy[rowIdx]
    channelIdx = currObj[3]

    currArr = markers_arr[channelIdx]
        
    currRow = np.array((currObj[0], currObj[1], currObj[2]), dtype=[('label', 'U64'), ('position', np.float32), ('duration', np.float32)])  # Create tuple (Label, Pos, Duration)
    # currArr.append(currRow)
    # Append the tuple to the numpy array
    currArr = np.append(currArr, currRow)
    
    # Update the array of the current channel    
    markers_arr[channelIdx] = currArr

print("Markers Array: ", markers_arr)

: 

###  Convert the array to a numpy array to allow writing to a .npy file

In [ ]:
# Convert the 2D array to a 1D numpy array
final_markers_npy = np.array(markers_arr, dtype=object)

preview_np_array(final_markers_npy, "final_markers_npy")

: 

### Write the processed markers into a .npy file

In [ ]:
file_name = f"{PATH_TO_FILE}seeg_{PATIENT_LABEL}_markers.npy"

EXPORT_MARKERS = True
if EXPORT_MARKERS:
    np.save(file_name, final_markers_npy)   # Save the data to a numpy file (not stored in git due to size)

: 

## See what is the average number of annotated events per channel

In [ ]:
ripple_counter = 0
fr_counter = 0

for marker in markers_npy:
    curr_label = marker[0]
    curr_pos = marker[1]

    if curr_label == 'Ripple':
        ripple_counter += 1
    elif curr_label == 'Fast Ripple':
        fr_counter += 1
    else:
        print(f"Unknown label: {curr_label}")

print(f"Ripple Count: {ripple_counter}")
print(f"Fast Ripple Count: {fr_counter}")

: 

In [ ]:
# Ripple Frequency per minute and per channel
ripple_freq = ripple_counter / ((input_duration / 60) * num_channels)
fr_freq = fr_counter / ((input_duration / 60) * num_channels)

print(f"Ripple Frequency: {ripple_freq} ripples / minute / channel")
print(f"Fast Ripple Frequency: {fr_freq} fast ripples / minute / channel")

: 

### Find the channels with the most annotated events

In [ ]:
# Build a dictionary with the channel idx and the number of annotated events
channel_event_count = {}
for idx in range(len(final_markers_npy)):
    channel_event_count[idx] = len(final_markers_npy[idx])
print("Channel Event Count: ", channel_event_count)

# Sort the dictionary by the number of events
sorted_channel_event_count = dict(sorted(channel_event_count.items(), key=lambda item: item[1], reverse=True))
print("Sorted Channel Event Count: ", sorted_channel_event_count)
# {ch_idx: num_events}

: 

---

## SEEG Data

In [ ]:
print(f"data: {data['data']}")

print(f"Shape of data: {data['data'].shape}")
# Shape of the data is (channels, samples)
# Nº of channels = 960 (Each channel represents a different electrode)
# Nº of samples = 245760

: 

### Shape of the data
The shape of the data is (n_channels, n_samples)
- Number of channels: 960
- Number of samples: 245760

The **data corresponds** to the recordings of the SEEG signals from 960 channels, acquiring a total of 245760 samples for each channel.

In [ ]:
recorded_data = data['data']

: 

Each value of the `recorded_data` is a float number that represents the amplitude of the signal at that specific time (voltage). The voltage is measured in millivolts (mV)?? TODO: Check units

# Change the structure of the data

Let's change the structure of the data to a 2D array that is ordered by time. This way, we can use the input data of various channels together by following the time order. 

Therefore, let's **transform the shape from (num_channels, num_samples) to (num_samples, num_channels)**. This way, each row will represent a time point and contains the voltage values of all channels at that time point. 

It is not necessary to specify the time of each row since there is a designated sampling rate of the input.

The structure is exemplified below, with a total of 245760 rows:

| Channel 1       | Channel 2     | Channel ...     | Channel 960   |
|-----------------|---------------|-----------------|---------------|
| 3               | 1             | 3               | 1             |
| 8               | 0             | 7               | 15            |
| ...             | ...           | ...             | ...           |
| 14              | 5             | 3               | 1             |


## Select the channels to be used
For the sake of simplicity, we can define a list of channels to be used.

In [ ]:
# channels_used: set = {1, 2, 3, 4, 5, 6, 7, 8}
channels_used = set(range(1, num_channels+1, 1))

print(channels_used)

: 

In [ ]:
ordered_recorded_data = recorded_data.T     # Swap the structure of the recorded_data to (num_samples, num_channels)

ordered_recorded_data.shape

: 

# Write the processed data to a .npy file

Finally, we write the processed data to a .npy file. This way, we can use it in the Spiking Neural Networks (SNN) model.

The .npy file is a binary file that contains the processed data in a numpy array format. This format is easy to read and write, and it is compatible with the numpy library.

In [ ]:
file_name = f"{PATH_TO_FILE}seeg_{PATIENT_LABEL}.npy"

EXPORT_SEEG = True
if EXPORT_SEEG:
    np.save(file_name, ordered_recorded_data)   # Save the data to a numpy file (not stored in git due to size)

: 